In [ ]:
# These are the Libraries Used
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
# Please Enter The Product To Search
search = input("Please Enter The Product To Search For : ")

Please Enter The Product To Search For : Samsung


In [ ]:
# Amazon don't allow to use its site if it is tool used is a browser so applying header. As I am using Google-Chrome so i have used a header for chrome
headers = {'User-Agent':'Chrome/55.0.2883.87'}

In [ ]:
# Number of pages to search for the current product
num_pages_to_search = int(input("Number of pages to search for the current product: "))

Number of pages to search for the current product: 2


In [ ]:
# Helper method to return the page requested for the product
def SearchProduct(string, number):
    # Makiing the url for the product
    url = "https://www.amazon.in/s?k="+ string +'&page=' + str(number)
    # Requesting the page
    page = requests.get(url, headers=headers)
    return page

In [ ]:
beautSoup = []
for i in range(1, num_pages_to_search + 1):
  # Creating the instance of BeautifulSoup For each page and saving in a list
  beautSoup.append(BeautifulSoup(SearchProduct(search, i).content))

In [ ]:
data = {}
# Looping through all the pages that we requested above
for beaut in beautSoup:
  # The products are in a div tag with the class s-include-content-margin s-border-bottom s-latency-cf-section so extracting all the products
  divs_cards = beaut.find_all('div', attrs={'class': "s-include-content-margin s-border-bottom s-latency-cf-section"})
  # Now looping through products div extracted above
  for i in divs_cards:
    # The link to the detail page of the prodcuct is in a Anchor tag with a class 'a-link-normal s-no-outline'
    a_link_norms = i.find('a', attrs={'class': 'a-link-normal s-no-outline'})
    # The symbol of the currency is in a span tag with a class a-price-symbol
    price_sym = i.find('span', attrs={'class': "a-price-symbol"})
    # Now wchecking if price_symbol is not None then only we will take the text of the tag
    if price_sym:
      price_sym = price_sym.text
    # The price of the product is in a span tag with a class a-price-whole
    price = i.find('span', attrs={'class': "a-price-whole"})
    # Now checking if price is not None then only we will take the text of the tag
    if price:
      price = price.text
    # Now extracting the title of the product which is inside span tag with a class 'a-size-medium a-color-base a-text-normal'
    title = i.find('span', attrs={"class": 'a-size-medium a-color-base a-text-normal'}).text
    # Now extracting the unique id of the product which is in a property of ['href'] of anchor tag
    asin = a_link_norms['href'].split("/")[-1].split("?")[0]
    # Now checking if we don't have the current asin value in our data-dictionary and price_sym and price are not None
    if asin not in data and price_sym is not None and price is not None:
      data[asin] = []
      data[asin].extend(["https://www.amazon.in/" + a_link_norms["href"], title, price_sym + price])
    else:
      continue
  
print(data)

{'B07DJCJBB3': ['https://www.amazon.in//Samsung-Galaxy-Mirage-128GB-Storage/dp/B07DJCJBB3?dchild=1', 'Samsung Galaxy M31s (Mirage Blue, 6GB RAM, 128GB Storage) - Get Flat Rs 2,500 Instant Discount with select bank cards - Limited Period Offer', '₹18,499'], 'B07HGJJ559': ['https://www.amazon.in//Samsung-Galaxy-Midnight-Blue-Storage/dp/B07HGJJ559?dchild=1', 'Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)', '₹12,499'], 'B07HGGYWL6': ['https://www.amazon.in//Samsung-Galaxy-Ocean-128GB-Storage/dp/B07HGGYWL6?dchild=1', 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage) - Get Flat Rs 2,500 Instant Discount with select bank cards - Limited Period Offer', '₹16,499'], 'B07DJCJBBN': ['https://www.amazon.in//Samsung-Galaxy-Mirage-128GB-Storage/dp/B07DJCJBBN?dchild=1', 'Samsung Galaxy M31s (Mirage Blue, 8GB RAM, 128GB Storage) - Get Flat Rs 2,500 Instant Discount with select bank cards - Limited Period Offer', '₹20,499'], 'B07HGH8D2R': ['https://www.amazon.in//Samsung-Galaxy-Raven-

In [ ]:
# Now asking for the number of pages through the reviews are to be fetched
no_of_reviews = int(input("Number Of Pages From Which Reviews To Fetch : "))
final_data = {}
# looping through all the products
for i in data:
  link = data[i][0]
  print(link)
  try:
    # requesting the product-detail page of the product
    page = requests.get(link, headers=headers)
    time.sleep(2)
  except:
    # print("Error Occured")
    continue
  # beautifulSoup instance of the particular page/product detail page
  beaut = BeautifulSoup(page.content)
  # rating of the product is in the span tag of data-hook attribute with name rating-out-of-text
  rating = beaut.find('span', attrs={'data-hook': "rating-out-of-text"})
  # Number of people rated of the product is in the div tag of data-hook attribute with name total-review-count
  globals = beaut.find('div', attrs={'data-hook': "total-review-count"})
  # getting the table which contains stars percentange (5% 4% 3% 2% 1%)
  table = beaut.find('table', attrs={'class': "a-normal a-align-center a-spacing-base"})
  # stars percentage are inside the span tag  with a class "a-size-base" within the table 
  stars = table.find_all('span', attrs={'class': "a-size-base"})
  star = [i.text.strip().strip("\n") for i in stars if i.text.strip().strip("\n") != '' and "%" in i.text.strip().strip("\n")] 
  if len(star) != 5:
    continue
  final_data[i] = []
  # Skeleton of the review link
  reviews_link = "https://www.amazon.in/{0}/product-reviews/{1}/ref=cm_cr_arp_d_paging_btm_next_{2}?ie=UTF8&reviewerType=all_reviews&pageNumber={2}"
  # Extracting the title of the product for review link
  title = link.split("https://www.amazon.in//")[1].split("/")[0]
  reviews = {}
  for idx in range(1, no_of_reviews + 1):
    try:
      # Reausting the page of the review and formatting the link with title, productId, pageNo.
      page_review = requests.get(reviews_link.format(title, i, idx), headers=headers)
      time.sleep(2)
    except:
      continue
    # print(reviews_link.format(title, i, idx))
    review_beaut = BeautifulSoup(page_review.content)
    # all the reviews are inside div with id cm_cr-review_list
    review_list = review_beaut.find('div', attrs={'id': "cm_cr-review_list"})
    # Taking all the review inside a list and a particular review is inside a div tag with class = a-section celwidget
    review_diff = review_list.find_all('div', attrs={'class': "a-section celwidget"})
    # Looping through all reviews of the current page
    for review in review_diff:
      # Extracting user profile from anchor tag with class a-profile 
      profile = review.find('a', attrs={'class': 'a-profile'})
      try:
        # Now extrating the profile Id
        profile_id = profile['href'].split(".")[-1]
      except:
        continue
      # Now checking if profile id is not present in the reviews of the current product
      if profile_id not in reviews:
        reviews[profile_id] = []
      else:
        continue
      # Now fetching review details
      try:
        name = profile.text
        rate = review.find('span', attrs={'class': 'a-icon-alt'}).text
        review_title = review.find('a', attrs={'data-hook': "review-title"}).text.strip("\n")
        review_date = review.find('span', attrs={'data-hook': "review-date"}).text.strip("\n")
      except:
        continue
      try:
        review_vote = review.find('span', attrs={'data-hook': "helpful-vote-statement"}).text.strip("\n")
      except:
        review_vote = "0 people found this helpful"
      try:
        review_specs = review.find('a', attrs={'data-hook': "format-strip"}).text.strip("\n")
      except:
        review_specs = "No Specs Found"
      try:
        review_body = review.find('span', attrs={'data-hook': "review-body"}).text.strip("\n")
      except:
        review_body = "No Body Found"

      print(profile_id)
      print(name)
      print(rate)
      print(review_title)
      print(review_date)
      print(review_specs)
      print(review_body)
      print(review_vote)
      print()
      print()
      reviews[profile_id].extend([name, rate, review_title, review_date, review_specs, review_body, review_vote])
  if rating and globals and stars and review_diff:
    final_data[i].extend(data[i][0: ])
    # appending all the details and reviews of the product
    final_data[i].extend([rating.text, globals.text.strip(), star, reviews])

Streaming output truncated to the last 5000 lines.
Size name: 4GB RAM, 64GB StorageColour: Blue
  Brilliant camera, huge battery life and brilliant display in addition with the premium feeling of SAMSUNG. It's a shame that this phone's launch was delayed due to lockdown.Highly Recommended phone 😊😊
1,912 people found this helpful


AGPSZTJVPVEEVLTA7OTBZWNJJEIA
Shobhanath
1.0 out of 5 stars
Feeling cheated
Reviewed in India on 20 May 2020
Size name: 4GB RAM, 64GB StorageColour: Blue
  Defective product received, it gets 8 to 12 hours in charging and Amazon is not exchanging the product
767 people found this helpful


AGBO5K36FD7YJUHJP7NC3LXB3LRA
Ravi Kumar
3.0 out of 5 stars
Battery life is not upto mark
Reviewed in India on 15 May 2020
Size name: 4GB RAM, 64GB StorageColour: Blue
  I am writing my review after using it for 6 days.I have bought sumsung phone after 7 years hoping it could be good phone but samsung again disappointed me a lot.They claim it has 6000mAH battery but drain lik

In [ ]:
# Removing profile (User Profile) from reviews where there was an empty list
for key, value in final_data.items():
  k = []
  for ky in value[6]:
    k.append(ky)
  for i in k:
    if len(value[6][i]) > 0:
      pass
    else:
      del value[6][i]
print(final_data)

{'B07DJCJBB3': ['https://www.amazon.in//Samsung-Galaxy-Mirage-128GB-Storage/dp/B07DJCJBB3?dchild=1', 'Samsung Galaxy M31s (Mirage Blue, 6GB RAM, 128GB Storage) - Get Flat Rs 2,500 Instant Discount with select bank cards - Limited Period Offer', '₹18,499', '4.3 out of 5', '28,040 global ratings', ['63%', '21%', '6%', '3%', '7%'], {'AEFKQOX3DV7JNJENI5JCL7MPAALA': ['J.G.Krishnaa', '5.0 out of 5 stars', 'HONEST M31s. HONEST PERFORMANCE. HONEST PRICE.', 'Reviewed in India on 10 August 2020', 'Colour: Mirage BlackStyle name: 6GB RAM', "  Honest review for a Honest Samsung phone. I am writing this review after using M31s for more than half month. And my verdict based on truly improved Samsung M series with my experience & tested after usage only.BUILD QUALITY. Superior, as it matches any S series flagship premium in classy look, strong built for this budget amount. My hand is small but still I found 6.5 inches display is good to handle as it is well managed for one hand use also.DISPLAY & FIN

In [ ]:
# Printing the information extracted
with open('Product_Detail_And_Review.txt', 'w') as file:
  for key, value in final_data.items():
    file.write("Product Id: {} \n".format(key))
    file.write("Product Title: {} \n".format(value[1]))
    file.write("Product Price: {} \n".format(value[2]))
    file.write("Overall Rating: {} \n".format(value[3]))
    file.write("People Rated: {} \n".format(value[4]))
    file.write("Percentage 5star: {} \n".format(value[5][0]))
    file.write("Percentage 4star: {} \n".format(value[5][1]))
    file.write("Percentage 3star: {} \n".format(value[5][2]))
    file.write("Percentage 2star: {} \n".format(value[5][3]))
    file.write("Percentage 1star: {} \n".format(value[5][4]))
    file.write("Product Link: {} \n".format(value[0]))
    file.write("\n")
    for review, val in value[6].items():
      file.write("=============== REVIEW OF CUSTOMER-ID: {} =================== \n".format(review))
      file.write("Customer Name: {} \n".format(val[0]))
      file.write("Customer Rating: {} \n".format(val[1]))
      file.write("Title of the review: {} \n".format(val[2]))
      file.write("Review Date: {} \n".format(val[3]))
      file.write("Specs:( {} ) \n".format(val[4]))
      file.write("Review Body: {}\n".format(val[5]))
      file.write("People Found This Review Helpful: {} \n".format(val[6]))
      file.write("\n")
    file.write("\n")
    file.write("========================= NEXT PRODUCT ==========================")
    